In [1]:
import pandas as pd
import numpy as np
import email
import os
from email.parser import Parser

In [2]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [84]:
# extract the information of email
# still need some works to get number of cc and bcc
def info_of_email(path,List_of_files,classification):
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    parser = Parser()
    info = []
    for files in List_of_files:
        emailText =  open(path + files, 'rb').read()
        email = parser.parsestr(emailText)
        if email.get('cc') == None:
            no_cc = 0
        else:
            no_cc = len(email.get('cc'))
        if email.get("bcc") == None:
            no_bcc = 0
        else:
            no_bcc = len(email.get('bcc'))
        Date = email.get("Date")
        try:
            Date = re.split(':|,| ',Date)
            info.append([Date[4], Date[3], Date[2], Date[0], Date[5], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), " ".join([i for i in walkMsg(email) if i != None])])
        except:
            info.append([[], [], [], [], [], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), " ".join([i for i in walkMsg(email) if i != None]), classification])
    return info

In [85]:
listing = os.listdir('./Spam_BG/')
listing = [i for i in listing if i[-3:] == "txt"]
email_df = pd.DataFrame(info_of_email('./Spam_BG/', listing, 'spam'),
                       columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [86]:
email_df.head()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,2004,Jul,31,Sat,12,"""Bertha "" <Denny@mailbox.sk>",rait@bruce-guenter.dyndns.org,0,0,multipart/alternative,Get it up and keep it up praecox,<html><head><meta http-equiv=Content-Type cont...,None
1,2004,Jul,31,Sat,22,"""LZEDEMCNODQBEZQBOV@encryption.com"" <Opal.Cham...",cvs@bruce-guenter.dyndns.org,29,0,multipart/alternative,Winner: $332573,<html>\nYour [m]ortgage application was approv...,None
2,2004,Aug,01,Sun,01,Connie Wong <tatypin@huhmail.com>,Rait <rait@bruce-guenter.dyndns.org>,0,0,multipart/alternative,Need software? Click here.,TOP quality software:<br><br>\n<b>Special Offe...,None
3,2004,Jul,30,Fri,17,shark@promotions.com <shark@promotions.com>,"""bruceg@em.ca"" <bruceg@em.ca>",0,0,multipart/alternative,Loyalty Bonuses for Shark Casino Players!,\n \n \n \n \n \n Featuring this weeken...,None
4,2004,Jul,31,Sat,06,"""Important! Search Engine Alert!"" <rockwelldat...","""Bruceg"" <bruceg@em.ca>",0,0,multipart/alternative,Your web site has dropped out of the Search En...,Rockwell Data Corp \n\nThis Email has been sen...,None


In [74]:
path = './Spam_BG/'
listing = os.listdir(path)
listing = [i for i in listing if i[-3:] == "txt"]

parser = Parser()
info = []
for files in listing:
    emailText =  open(path + files, 'rb').read()
    email = parser.parsestr(emailText)
    if email.get('cc') == None:
        no_cc = 0
    else:
        no_cc = len(email.get('cc'))
    if email.get("bcc") == None:
        no_bcc = 0
    else:
        no_bcc = len(email.get('bcc'))
    Date = email.get("Date")
    try:
        Date = re.split(':|,| ',Date)
        info.append([Date[4], Date[3], Date[2], Date[0], Date[5], email.get("From"), 
                     email.get('To'), no_cc, no_bcc, 
                     str(email.get("Content-Type")).split(";")[0],
                     email.get('subject'), " ".join([i for i in walkMsg(email) if i != None])])
    except:
        info.append([[], [], [], [], [], email.get("From"), 
                     email.get('To'), no_cc, no_bcc, 
                     str(email.get("Content-Type")).split(";")[0],
                     email.get('subject'), " ".join([i for i in walkMsg(email) if i != None])])

,0,1,2,3,4,5,6,7,8,9,10,11
0,2004,Jul,31,Sat,12,"""Bertha "" <Denny@mailbox.sk>",rait@bruce-guenter.dyndns.org,0,0,multipart/alternative,Get it up and keep it up praecox,<html><head><meta http-equiv=Content-Type cont...
1,2004,Jul,31,Sat,22,"""LZEDEMCNODQBEZQBOV@encryption.com"" <Opal.Cham...",cvs@bruce-guenter.dyndns.org,29,0,multipart/alternative,Winner: $332573,<html>\nYour [m]ortgage application was approv...
2,2004,Aug,01,Sun,01,Connie Wong <tatypin@huhmail.com>,Rait <rait@bruce-guenter.dyndns.org>,0,0,multipart/alternative,Need software? Click here.,TOP quality software:<br><br>\n<b>Special Offe...
3,2004,Jul,30,Fri,17,shark@promotions.com <shark@promotions.com>,"""bruceg@em.ca"" <bruceg@em.ca>",0,0,multipart/alternative,Loyalty Bonuses for Shark Casino Players!,\n \n \n \n \n \n Featuring this weeken...
4,2004,Jul,31,Sat,06,"""Important! Search Engine Alert!"" <rockwelldat...","""Bruceg"" <bruceg@em.ca>",0,0,multipart/alternative,Your web site has dropped out of the Search En...,Rockwell Data Corp \n\nThis Email has been sen...
5,2004,Aug,01,Sun,05,"""Louis Henrietta"" <ndqpgk4ug@ioc.net>",bruceg@em.ca,0,0,multipart/alternative,"WE SHIP U MICR0S0FT, AD0BE, N0RTON & 80 MORE S...",become grabbed dont start \nexcuse sound nay \...
6,2004,Jul,31,Sat,01,"""Infor Info"" <infor_123@ozu.es>",infor_123@ozu.es,0,0,text/plain,Compliment.,"Dear Friend,\n\n REQUEST FOR URGENT BUSINESS ..."
7,2004,Jul,31,Sat,19,"""Roberto Lange"" <xaotljzjaho@hotmail.com>",bait@em.ca,0,0,multipart/alternative,about the account,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
8,2004,Aug,01,Sun,22,"""Harriet Mcgee"" <therein2@pinghetang.com>","""Bruceg"" <bruceg@em.ca>",0,0,multipart/alternative,Super-discounts on Ambien and Soma,<html>\n<title> basalt deducible oaken fafnir ...
9,2004,Jul,31,Sat,10,"""Brian_loebig"" <Jikkloscapegoat@africamail.com>",bruceg@em.ca,0,0,text/html,e`n`1`a`r`g`e your p`e`n`1`s whittli...,"<html>\n<font color=\n\n""#FFFFFB"">broadside ou..."


In [40]:
Date[4].split(":")

['12', '39', '33']

In [42]:
email.get('cc')

In [98]:
re.findall(r'\<\>',email.get("From"))

[]

In [ ]:
r'\[([^]]*)\]'